<a href="https://colab.research.google.com/github/Ehsunpy/Graph-RAG/blob/main/Graph_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 6.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("dongx1997/NutriBench", "v2")

In [ ]:
import pandas as pd
df = pd.DataFrame(ds['train'])

In [ ]:
df.head()

,meal_description,carb,fat,energy,protein,country,serving_type
0,"For breakfast, I ate a plain bun weighing 126 ...",90.8,4.2,439.0,9.6,ZMB,metric
1,"For lunch, I had 171 grams of boiled fresh gro...",97.8,37.8,806.0,27.9,ZMB,metric
2,"I’ve got 9 grams of boiled kasepa fish, 34 gra...",51.2,14.2,363.0,8.7,ZMB,metric
3,I had a breakfast consisting of 51g of raw whi...,163.8,8.9,792.0,19.0,ZMB,metric
4,"For dinner, I had 105g of raw maize flour alon...",81.1,4.6,390.0,8.4,ZMB,metric


In [ ]:
df.shape

(15617, 7)

In [ ]:
df.columns

Index(['meal_description', 'carb', 'fat', 'energy', 'protein', 'country',
       'serving_type'],
      dtype='object')

In [ ]:
df.dtypes

,0
meal_description,object
carb,float64
fat,float64
energy,float64
protein,float64
country,object
serving_type,object


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15617 entries, 0 to 15616
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   meal_description  15617 non-null  object 
 1   carb              15617 non-null  float64
 2   fat               15617 non-null  float64
 3   energy            15617 non-null  float64
 4   protein           15617 non-null  float64
 5   country           15617 non-null  object 
 6   serving_type      15617 non-null  object 
dtypes: float64(4), object(3)
memory usage: 854.2+ KB


In [ ]:
for i in range(5):
  print(df.iloc[i])
  print("*"*30)

meal_description    For breakfast, I ate a plain bun weighing 126 ...
carb                                                             90.8
fat                                                               4.2
energy                                                          439.0
protein                                                           9.6
country                                                           ZMB
serving_type                                                   metric
Name: 0, dtype: object
******************************
meal_description    For lunch, I had 171 grams of boiled fresh gro...
carb                                                             97.8
fat                                                              37.8
energy                                                          806.0
protein                                                          27.9
country                                                           ZMB
serving_type                        

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm

df['country'] = df['country'].fillna('Unknown')
df['serving_type'] = df['serving_type'].fillna('Unknown')

print("Loading embedding model...")
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['meal_description'].tolist(), show_progress_bar=True)

df['embedding'] = embeddings.tolist()

print("Embeddings for Neo4j import.")

Loading embedding model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/489 [00:00<?, ?it/s]

KeyboardInterrupt: 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 10.7 MB/s eta 0:00:00
Loading model for search...

--- تست ۱: جستجوی متنی ساده ---


ClientError: {neo4j_code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.vector.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such vector schema index: meal_description_vector} {gql_status: 52N37} {gql_status_description: error: procedure exception - procedure execution error. Execution of the procedure db.index.vector.queryNodes() failed.}

In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm
# use neo4j database

URI = "neo4j+s://91fb5ba0.databases.neo4j.io"
AUTH = ("neo4j", "zk2xfyheSB4-FJ-JM4pOnsVkrzTzIar0mTxdkSqALwI")

df['id'] = df.index.astype(str)

def setup_schema(driver):
    print("Setting up constraints and indexes...")

    # الف) ساخت ایندکس وکتور (برای جستجوی معنایی) - ابعاد 384 برای مدل MiniLM
    driver.execute_query("""
        CREATE VECTOR INDEX meal_description_vector IF NOT EXISTS
        FOR (m:Meal) ON (m.embedding)
        OPTIONS {indexConfig: {
            `vector.dimensions`: 384,
            `vector.similarity_function`: 'cosine'
        }}
    """)

    # ب) ساخت محدودیت یکتایی برای سرعت بیشتر و جلوگیری از تکرار
    driver.execute_query("CREATE CONSTRAINT meal_id IF NOT EXISTS FOR (m:Meal) REQUIRE m.id IS UNIQUE")
    driver.execute_query("CREATE CONSTRAINT country_name IF NOT EXISTS FOR (c:Country) REQUIRE c.name IS UNIQUE")
    driver.execute_query("CREATE CONSTRAINT serving_name IF NOT EXISTS FOR (s:ServingType) REQUIRE s.name IS UNIQUE")
    print("Schema setup complete.")

# 3. کوئری سایفر برای ایمپورت داده
import_query = """
UNWIND $rows AS row

// ساخت نود غذا با ID مشخص
MERGE (m:Meal {id: row.id})
SET m.description = row.meal_description,
    m.carb = row.carb,
    m.fat = row.fat,
    m.energy = row.energy,
    m.protein = row.protein,
    m.embedding = row.embedding

// ایجاد رابطه با کشور (اگر Unknown نباشد)
FOREACH (_ IN CASE WHEN row.country <> 'Unknown' THEN [1] ELSE [] END |
    MERGE (c:Country {name: row.country})
    MERGE (m)-[:ORIGINATES_FROM]->(c)
)

// ایجاد رابطه با نوع سرو (اگر Unknown نباشد)
FOREACH (_ IN CASE WHEN row.serving_type <> 'Unknown' THEN [1] ELSE [] END |
    MERGE (s:ServingType {name: row.serving_type})
    MERGE (m)-[:SERVED_AS]->(s)
)
"""

def batch_import(tx, rows):
    tx.run(import_query, rows=rows)

print("Connecting to Neo4j AuraDB...")
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        setup_schema(driver)

        data_dict = df.to_dict('records')
        batch_size = 1000
        print(f"Starting import of {len(data_dict)} records...")

        with driver.session() as session:
            for i in tqdm(range(0, len(data_dict), batch_size), desc="Uploading Batches"):
                batch = data_dict[i:i+batch_size]
                try:
                    session.execute_write(batch_import, batch)
                except Exception as e:
                    print(f"Error in batch starting at index {i}: {e}")

    print("\n✅ Import finished successfully! Database is ready.")

except Exception as e:
    print(f"Connection failed: {e}")

Connecting to Neo4j AuraDB...
Setting up constraints and indexes...
Schema setup complete.
Starting import of 15617 records...


Uploading Batches: 100%|██████████| 16/16 [00:30<00:00,  1.93s/it]


✅ Import finished successfully! Database is ready.


In [ ]:
from neo4j import GraphDatabase

URI = "neo4j+s://91fb5ba0.databases.neo4j.io"
AUTH = ("neo4j", "zk2xfyheSB4-FJ-JM4pOnsVkrzTzIar0mTxdkSqALwI")

index_query = """
CREATE VECTOR INDEX meal_description_vector IF NOT EXISTS
FOR (m:Meal)
ON (m.embedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: 384,
 `vector.similarity_function`: 'cosine'
}}
"""

print("Connecting to create index...")
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        # بررسی اتصال
        driver.verify_connectivity()

        with driver.session() as session:
            print("⏳ Creating Vector Index (this might take a moment)...")
            session.run(index_query)
            print("✅ Command sent successfully.")

            print("\nVerifying index status:")
            result = session.run("SHOW VECTOR INDEXES YIELD name, state, populationPercent")
            found = False
            for record in result:
                if record["name"] == "meal_description_vector":
                    found = True
                    print(f"- Index Name: {record['name']}")
                    print(f"- State: {record['state']}") # باید ONLINE یا POPULATING باشد
                    print(f"- Population: {record['populationPercent']}%")

            if not found:
                print("⚠️ Warning: Index specifically named 'meal_description_vector' was not found in the list.")

except Exception as e:
    print(f"❌ Error creating index: {e}")

Connecting to create index...
⏳ Creating Vector Index (this might take a moment)...
✅ Command sent successfully.

Verifying index status:
- Index Name: meal_description_vector
- State: ONLINE
- Population: 100.0%


In [ ]:
from neo4j import GraphDatabase

URI = "neo4j+s://91fb5ba0.databases.neo4j.io"
AUTH = ("neo4j", "zk2xfyheSB4-FJ-JM4pOnsVkrzTzIar0mTxdkSqALwI")

print("Checking database count...")

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        with driver.session() as session:
            count = session.run("MATCH (m:Meal) RETURN count(m) as c").single()["c"]

            print(f"\n📊 Total Meals in Graph: {count}")

            if count > 0:
                print("✅ Insert database")
            else:
                print("⚠️ data base is empty")

except Exception as e:
    print(f"❌ Connection Error: {e}")

Checking database count...

📊 Total Meals in Graph: 15617
✅ دیتابیس با موفقیت پر شده است.


this for runn when database is ready

In [ ]:
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer

URI = "neo4j+s://91fb5ba0.databases.neo4j.io"
AUTH = ("neo4j", "zk2xfyheSB4-FJ-JM4pOnsVkrzTzIar0mTxdkSqALwI")
DATABASE = "neo4j"

print("Loading model for search...")
model = SentenceTransformer('all-MiniLM-L6-v2')

def graph_rag_search(query_text, min_protein=None, max_fat=None, limit=3):
    query_vector = model.encode(query_text).tolist()

    cypher_query = """
    CALL db.index.vector.queryNodes('meal_description_vector', 50, $embedding)
    YIELD node AS m, score
    WHERE ($min_protein IS NULL OR m.protein >= $min_protein)
      AND ($max_fat IS NULL OR m.fat <= $max_fat)
    OPTIONAL MATCH (m)-[:ORIGINATES_FROM]->(c:Country)
    OPTIONAL MATCH (m)-[:SERVED_AS]->(s:ServingType)
    RETURN m.description AS description,
           m.protein AS protein,
           m.fat AS fat,
           m.energy AS energy,
           c.name AS country,
           s.name AS serving_type,
           score
    ORDER BY score DESC
    LIMIT $limit
    """

    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database=DATABASE) as session:
            result = session.run(
                cypher_query,
                embedding=query_vector,
                min_protein=min_protein,
                max_fat=max_fat,
                limit=limit
            )
            return [record.data() for record in result]

print("teset1 ")
results = graph_rag_search("Pasta with tomato sauce")
for r in results:
    print(f"Food: {r['description'][:50]}... | Country: {r['country']} | Score: {r['score']:.3f}")

print("test 2 ")
results = graph_rag_search("Pasta", min_protein=20.0)
for r in results:
    print(f"Food: {r['description'][:50]}... | Protein: {r['protein']} | Score: {r['score']:.3f}")


Loading model for search...
teset1 
Food: For dinner, I'm having a cup of pasta with a homem... | Country: USA | Score: 0.919
Food: For lunch, I'm having a cup of pasta with tomato-b... | Country: USA | Score: 0.912
Food: For dinner, I'm having 250 grams of pasta with a h... | Country: USA | Score: 0.898
test 2 
Food: I'm heating up a can of pasta with tomato sauce an... | Protein: 24.4 | Score: 0.784


In [ ]:
!pip install langchain-openai
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


OPENROUTER_API_KEY = "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"

llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    model="google/gemma-3-12b-it:free",
    temperature=0,
)

# 2. اصلاح پرامپت (System Prompt را حذف و در User ادغام می‌کنیم)
# به جای جدا کردن system و user، همه را در یک قالب می‌ریزیم
full_template = """
[Instructions]
تو یک دستیار متخصص تغذیه و سلامت هستی.
من به تو اطلاعات دقیق استخراج شده از پایگاه داده را می‌دهم (Context).
وظیفه تو این است که فقط و فقط بر اساس این اطلاعات به سوال کاربر پاسخ دهی.

قوانین مهم:
1. اگر در Context اطلاعاتی نبود، بگو "اطلاعاتی در دیتابیس ندارم".
2. از دانش عمومی خودت برای ساختن اعداد تغذیه‌ای استفاده نکن.
3. پاسخ را کاملا فارسی، روان و ساختاریافته بنویس.
4. حتما اسم غذا، کشور و مقادیر مغذی را ذکر کن.

[Context Data]
{context}

[User Question]
{question}
"""

# اینجا از from_template استفاده می‌کنیم که همه را به عنوان یک پیام User می‌فرستد
prompt_template = ChatPromptTemplate.from_template(full_template)

# ساخت زنجیره
chain = prompt_template | llm | StrOutputParser()

# 3. تابع اصلی (بدون تغییر در منطق، فقط استفاده از زنجیره جدید)
def ask_nutritionist_gemma(user_question):
    print(f"\n👤 سوال کاربر: {user_question}")

    # تشخیص پارامترها
    min_prot = None
    max_fat = None
    if "پروتئین بالا" in user_question or "high protein" in user_question:
        min_prot = 15.0
    if "کم چرب" in user_question or "low fat" in user_question:
        max_fat = 10.0

    print("🔍 در حال جستجو در گراف (Neo4j)...")
    # فرض بر این است که graph_rag_search از قبل تعریف شده است
    results = graph_rag_search(user_question, min_protein=min_prot, max_fat=max_fat, limit=4)

    if not results:
        return "❌ متاسفانه غذایی با این مشخصات در دیتابیس پیدا نشد."

    # آماده‌سازی متن
    context_str = ""
    for i, r in enumerate(results):
        context_str += f"""
        [گزینه {i+1}]
        نام غذا: {r['description']}
        کشور: {r['country']}
        پروتئین: {r['protein']} گرم
        چربی: {r['fat']} گرم
        انرژی: {r['energy']} کیلوکالری
        """

    print("🤖 در حال تولید پاسخ با Gemma 3...")
    # اجرای زنجیره
    response = chain.invoke({"context": context_str, "question": user_question})
    return response

# --- تست نهایی ---
print("✅ سیستم اصلاح شده آماده است.")
# تست کنید:
question = "یک غذای برای کاهش وزن پیشنهاد بده با دستور پختن آن"
print(ask_nutritionist_gemma(question))


question = " یک غذای برای کاهش وزن پیشنهاد بده با دستور پختن آن و وزن به همراه کالری ان"
print(ask_nutritionist_gemma(question))

✅ سیستم اصلاح شده آماده است.

👤 سوال کاربر: یک غذای برای کاهش وزن پیشنهاد بده با دستور پختن آن
🔍 در حال جستجو در گراف (Neo4j)...
🤖 در حال تولید پاسخ با Gemma 3...
با توجه به اطلاعات موجود در پایگاه داده، غذای "Red Gram Dhal Bhaji" همراه با برنج (Rice) از کشور هند می‌تواند گزینه مناسبی برای کاهش وزن باشد.

**اطلاعات تغذیه‌ای:**

*   **نام غذا:** Red Gram Dhal Bhaji و برنج (Rice)
*   **کشور:** هند (IND)
*   **پروتئین:** 2.1 گرم
*   **چربی:** 4.999999999999999 گرم
*   **انرژی:** 102.0 کیلوکالری

**دستور پخت (بر اساس اطلاعات موجود):**

متاسفانه دستور پخت دقیق این غذا در پایگاه داده موجود نیست. اما با توجه به نام غذا، می‌توان حدس زد که شامل موارد زیر است:

*   **Red Gram Dhal:** عدس قرمز
*   **Bhaji:** سبزیجات (نوع سبزیجات مشخص نیست)
*   **Rice:** برنج

برای تهیه این غذا می‌توانید از دستور پخت‌های رایج عدس و سبزیجات در آشپزی هندی استفاده کنید و آن را با برنج سرو کنید.

👤 سوال کاربر:  یک غذای برای کاهش وزن پیشنهاد بده با دستور پختن آن و وزن به همراه کالری ان
🔍 در حال جستجو در گراف (Neo4j)...